In [ ]:

import time
import pandas as pd
import keepa
from google.colab import drive
import os
#important accesskey must be define for it  to work
api = keepa.Keepa(accesskey)

# Dictionary of products


# Initialize an empty list to store dataframes
dataframes = []

failing_stores_dict = {}

for store_dict in to_extract:
    try:
        if api.update_status()["tokensLeft"] <= 1:
            print(f"Pausing for {api.time_to_refill+61} seconds...")
            time.sleep(api.time_to_refill+61)

        products = api.query(store_dict["query"],domain="DE",rating=True,buybox=True,days=542)

        missing_data = {}
        try:
            brand = products[0]['brand']
        except:
            brand = 'No api brand'
        try:
            product_group = [0]['productGroup']
        except:
            product_group = 'No api group'
        try:
            api_store_name = products[0]['brandStoreName']
        except:
            api_store_name = 'No api store name'
        try:
            df_new = products[0]["data"]["df_NEW"].reset_index()
            df_new.rename(columns={"value": "lowest_internet_price"}, inplace=True)
        except KeyError:
            df_new = pd.DataFrame()
            missing_data["df_NEW"] = True

        try:
            df_amazon = products[0]["data"]["df_AMAZON"].reset_index()
            df_amazon.rename(columns={"value": "amazon_price"}, inplace=True)
        except KeyError:
            df_amazon = pd.DataFrame()
            missing_data["df_AMAZON"] = True

        try:
            df_list_price = products[0]['data']['df_LISTPRICE'].reset_index()
            df_list_price.rename(columns={"value": "amazon_listed_price"}, inplace=True)
        except KeyError:
            df_list_price = pd.DataFrame()
            missing_data["df_LISTPRICE"] = True
        try:
            df_count_reviews = products[0]["data"]["df_COUNT_REVIEWS"].reset_index()
            df_count_reviews.rename(columns={"value": "total_count_reviews_raw"}, inplace=True)
            df_count_reviews["total_count_reviews_raw"] = df_count_reviews["total_count_reviews_raw"].fillna(method="ffill")
        except KeyError:
            df_count_reviews = pd.DataFrame()
            missing_data["df_COUNT_REVIEWS"] = True

        try:
            df_rating = products[0]["data"]["df_RATING"].reset_index()
            df_rating.rename(columns={"value": "avg_rating"}, inplace=True)
        except KeyError:
            df_rating = pd.DataFrame()
            missing_data["df_RATING"] = True
        try:
            df_buy_box = products[0]["data"]["df_BUY_BOX_SHIPPING"].reset_index()
            df_buy_box.rename(columns={"value": "buy_box_price"}, inplace=True)
        except KeyError:
            df_buy_box = pd.DataFrame()
            missing_data["df_BUY_BOX_SHIPPING"] = True
        try:
            df_sales_rank = products[0]["data"]["df_SALES"].reset_index()
            df_sales_rank.rename(columns={"value": "store_rank"}, inplace=True)
        except KeyError:
            df_sales_rank = pd.DataFrame()
            missing_data["df_SALES"] = True
        print(f"Store: {store_dict['store']}")
        print("df_new columns:", df_new.columns.tolist())
        print("df_amazon columns:", df_amazon.columns.tolist())
        print("df_count_reviews columns:", df_count_reviews.columns.tolist())
        print("df_rating columns:", df_rating.columns.tolist())
        print("df_buy_box columns:", df_buy_box.columns.tolist())
        print("df_sales_rank columns:", df_sales_rank.columns.tolist())


        merged_df = pd.merge(df_new, df_amazon, on="index", how="outer", suffixes=('_new', '_amazon'))

        merged_df = pd.merge(merged_df, df_count_reviews, on="index", how="outer", suffixes=('', '_reviews'))
        merged_df = pd.merge(merged_df, df_rating, on="index", how="outer", suffixes=('', '_rating'))

        merged_df = pd.merge(merged_df, df_buy_box, on="index", how="outer", suffixes=('', '_buy_box'))

        merged_df = pd.merge(merged_df, df_sales_rank, on="index", how="outer", suffixes=('', '_sales_rank'))
        merged_df = pd.merge(merged_df,df_list_price, on="index",how="outer",suffixes=('','_listed_price') )
        merged_df.rename(columns={"index": "date"}, inplace=True)

        merged_df["store"] = store_dict['store']
        merged_df["api store name"] = api_store_name
        merged_df["brand"] = brand
        merged_df["product group"] = product_group
        merged_df["product short name"] = store_dict["product short name"]
        merged_df["product full name"] = store_dict["product full name"]
        merged_df["deparment"] = store_dict["deparment"]
        merged_df["category"] = store_dict["category"]
        merged_df["query"] = store_dict["query"]
        merged_df.drop_duplicates(inplace=True)
        dataframes.append(merged_df)

        if missing_data:
            failing_stores_dict[store_dict['store']] = {"missing_data": missing_data, **store_dict}
        folder_name = store_dict['store']
        base_path = "/content/drive/MyDrive/keepa_data"
        csv_file = str(store_dict['store']) + " " + str(store_dict['query']) +".csv"

        folder_path = os.path.join(base_path, folder_name)
        csv_path = os.path.join(folder_path, csv_file)
        os.makedirs(folder_path, exist_ok=True)
        merged_df.to_csv(csv_path,index=False)
        old_df = pd.read_csv("/content/drive/MyDrive/keepa_data/safe_guard_df.csv")
        combined = pd.concat([old_df,merged_df]).reset_index(drop=True)
        combined.to_csv("/content/drive/MyDrive/keepa_data/safe_guard_df.csv",index=False)
    except Exception as e:
        print(f"Error processing {store_dict['store']}: {e}")
        failing_stores_dict[store_dict['store']] = {"error": str(e), **store_dict}

final_df = pd.concat(dataframes, ignore_index=True)
base_path = "/content/drive/MyDrive/keepa_data"

display(final_df)
final_df.to_csv("/content/drive/MyDrive/keepa_data/final_df_other_stores.csv")
concat_df = pd.read_csv('/content/drive/MyDrive/keepa_data/final_df.csv')
concat_df.drop(columns=['Unnamed: 0'], inplace=True)
update_final_df = pd.concat([final_df, concat_df], ignore_index=True)
update_final_df.to_csv("/content/drive/MyDrive/keepa_data/final_df.csv")
print("Failing stores dictionary:", failing_stores_dict)
